In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

from utils import *
from tqdm import tqdm

dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
d = 3
m = 8
t = 200
n = 30000
snr_min = 0
snr_max = 30
lamda = 0.2
radius = 0.1
mc_range = 3
coherent = False

array = UCA(m, lamda)
array.build_array(radius=0.1)
array.build_array_manifold()
array.build_transform_matrices_from_array_manifold()

observations, angles = generate_data(n, t, d, snr_min, snr_max, array, mc_range, coherent)

In [3]:
class my_model(nn.Module):

    def __init__(self, m: int, d: int, array, device: str = dev):
        
        super().__init__()
        self.m = m
        self.d = d
        self.array = array

        self.bn = nn.BatchNorm1d(2*self.m, device=device)
        self.rnn = nn.GRU(input_size=2*self.m, hidden_size=2*self.m, num_layers=1, batch_first=True, device=device)
        self.fc = nn.Linear(in_features=2*self.m, out_features=2*self.m*self.m, device=device)
        self.mlp = nn.Sequential(nn.Linear(in_features=array.nbSamples_spectrum, out_features=2*self.m, device=device), nn.ReLU(),
                                 nn.Linear(in_features=2*self.m, out_features=2*self.m, device=device), nn.ReLU(),
                                 nn.Linear(in_features=2*self.m, out_features=2*self.m, device=device), nn.ReLU(),
                                 nn.Linear(in_features=2*self.m, out_features=self.d, device=device))

    def forward(self, X: torch.Tensor):
        
        X = torch.cat((torch.real(X), torch.imag(X)), dim=-1)
        X = self.bn(X.transpose(1, 2)).transpose(1, 2)
        _, X = self.rnn(X)

        cov = self.fc(X[-1])
        cov = cov.reshape(-1, 2, self.m, self.m)
        cov = cov[:, 0, :, :] + 1j * cov[:, 1, :, :]
        cov = cov @ cov.conj().transpose(1, 2)
        vals, vecs = torch.linalg.eigh(cov)
        idx = torch.sort(torch.abs(vals), dim=1)[1].unsqueeze(dim=1).repeat(repeats=(1, self.m, 1))
        vecs = torch.gather(vecs, dim=2, index=idx)
        En = vecs[:, :, :(self.m - self.d)]
        
        y = get_spectrum(En, array=self.array)
        theta = self.mlp(y)
        
        return theta

In [4]:
nbEpoches = 200
lr = 1e-2
wd = 1e-8
batchSize = 512

x_train, x_valid, theta_train, theta_valid = train_test_split(observations, angles, test_size=0.2)
x_train, x_test, theta_train, theta_test = train_test_split(x_train, theta_train, test_size=0.2)

train_set = DATASET(x_train, theta_train)
valid_set = DATASET(x_valid, theta_valid)
test_set = DATASET(x_test, theta_test)

train_loader = DataLoader(train_set, batch_size=batchSize, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batchSize, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batchSize, shuffle=False)

In [5]:
model = my_model(m, d, array)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.2)
Loss, Loss_theta, Val_theta = [], [], []
bestVal = 1000

for i in tqdm(range(nbEpoches)):
    # Train
    running_loss = 0.0
    for data in train_loader:
        X, theta_true = data[0].to(dev), data[1].to(dev)
        optimizer.zero_grad()
        theta_pred = model(X)
        loss = RMSPE(theta_pred, theta_true) 
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    Loss.append(running_loss/len(train_loader))

    # scheduler.step()

    # Validation 
    with torch.no_grad():
        running_loss = 0.0
        for data in valid_loader:
            X, theta_true = data[0].to(dev), data[1].to(dev)
            theta_pred = model(X)
            loss = RMSPE(theta_pred, theta_true)
            running_loss += loss.item()
        
        Val_theta.append(running_loss/len(test_loader))

        if Val_theta[i] < bestVal:
            bestVal = Val_theta[i]
            torch.save(model.state_dict(), 'model_2.pt')

    print("Iteration {}: Loss theta = {}".format(i, Loss[-1]))

  0%|          | 1/200 [00:03<12:42,  3.83s/it]

Iteration 0: Loss theta = 0.4850638587223856


  1%|          | 2/200 [00:07<12:09,  3.69s/it]

Iteration 1: Loss theta = 0.43088793597723307


  2%|▏         | 3/200 [00:10<11:56,  3.64s/it]

...

Iteration 197: Loss theta = 0.061920609129102605


100%|█████████▉| 199/200 [11:58<00:03,  3.60s/it]

Iteration 198: Loss theta = 0.06082133193941493


100%|██████████| 200/200 [12:01<00:00,  3.61s/it]

Iteration 199: Loss theta = 0.06467727777597151


In [6]:
model_test = my_model(m, d, array)
model_test.load_state_dict(torch.load('model_2.pt', weights_only=True))
running_loss = 0.0 

with torch.no_grad():
    for data in test_loader:
        X, theta_true= data[0].to(dev), data[1].to(dev)
        theta_pred = model_test(X)
        loss = RMSPE(theta_pred, theta_true)
        running_loss += loss.item()

Acc_theta = running_loss/len(test_loader)

print("RMSE DoA =", Acc_theta)

RMSE DoA = 0.06337683498859406


In [7]:
array.x = array.x.cpu()
array.y = array.y.cpu()

observations, angles = generate_data(100, t, d, 0, 0, array, mc_range, coherent)
Acc_theta_0 = RMSPE(model_test(observations.to(dev)), angles)

print("0dB RMSE DoA =", Acc_theta_0)


angles_rare = []
for i in range(observations.shape[0]):
    angles_rare.append(RARE(observations[i].T, d, 3, array)[0])
angles_rare = torch.stack(angles_rare, dim=0)

print("RMSPE RARE =", RMSPE(angles_rare.cuda(), angles.cuda()))

0dB RMSE DoA = tensor(0.1843, device='cuda:0', grad_fn=<MeanBackward0>)
RMSPE RARE = tensor(0.3154, device='cuda:0')


In [8]:
array.x = array.x.cpu()
array.y = array.y.cpu()

observations, angles = generate_data(100, t, d, 5, 5, array, mc_range, coherent)
Acc_theta_5 = RMSPE(model_test(observations.to(dev)), angles)

print("5dB RMSE DoA =", Acc_theta_5)

angles_rare = []
for i in range(observations.shape[0]):
    angles_rare.append(RARE(observations[i].T, d, 3, array)[0])
angles_rare = torch.stack(angles_rare, dim=0)

print("RMSPE RARE =", RMSPE(angles_rare.cuda(), angles.cuda()))

5dB RMSE DoA = tensor(0.0798, device='cuda:0', grad_fn=<MeanBackward0>)
RMSPE RARE = tensor(0.1821, device='cuda:0')


In [9]:
array.x = array.x.cpu()
array.y = array.y.cpu()

observations, angles = generate_data(100, t, d, 10, 10, array, mc_range, coherent)
Acc_theta_10 = RMSPE(model_test(observations.to(dev)), angles)

print("10dB RMSE DoA =", Acc_theta_10)

angles_rare = []
for i in range(observations.shape[0]):
    angles_rare.append(RARE(observations[i].T, d, 3, array)[0])
angles_rare = torch.stack(angles_rare, dim=0)

print("RMSPE RARE =", RMSPE(angles_rare.cuda(), angles.cuda()))

10dB RMSE DoA = tensor(0.0584, device='cuda:0', grad_fn=<MeanBackward0>)
RMSPE RARE = tensor(0.1343, device='cuda:0')


In [10]:
array.x = array.x.cpu()
array.y = array.y.cpu()

observations, angles = generate_data(100, t, d, 15, 15, array, mc_range, coherent)
Acc_theta_15 = RMSPE(model_test(observations.to(dev)), angles)

print("15dB RMSE DoA =", Acc_theta_15)

angles_rare = []
for i in range(observations.shape[0]):
    angles_rare.append(RARE(observations[i].T, d, 3, array)[0])
angles_rare = torch.stack(angles_rare, dim=0)

print("RMSPE RARE =", RMSPE(angles_rare.cuda(), angles.cuda()))

15dB RMSE DoA = tensor(0.0603, device='cuda:0', grad_fn=<MeanBackward0>)
RMSPE RARE = tensor(0.0820, device='cuda:0')


In [11]:
array.x = array.x.cpu()
array.y = array.y.cpu()

observations, angles = generate_data(100, t, d, 20, 20, array, mc_range, coherent)
Acc_theta_15 = RMSPE(model_test(observations.to(dev)), angles)

print("15dB RMSE DoA =", Acc_theta_15)

angles_rare = []
for i in range(observations.shape[0]):
    angles_rare.append(RARE(observations[i].T, d, 3, array)[0])
angles_rare = torch.stack(angles_rare, dim=0)

print("RMSPE RARE =", RMSPE(angles_rare.cuda(), angles.cuda()))

15dB RMSE DoA = tensor(0.0580, device='cuda:0', grad_fn=<MeanBackward0>)
RMSPE RARE = tensor(0.0613, device='cuda:0')
